In [17]:
import pandas as pd

In [18]:
meta_muts = pd.read_pickle('./targ_mut_df.pkl')
display(len(meta_muts), meta_muts.head())

46

,exp,sample,Position,Mutation Type,Sequence Change,Details,Gene,ALEdb source,genetic,clean genes,bnums,genes of interest
468518,GLU_16321-M9-NC_000913_3gb-stationary-37-Gluco...,11 20 0 1,3041632,SNP,T→A,V107E (GTG→GAG),ygfZ,private,True,{ygfZ},{b2898},{b2898}
468601,GLU_16321-M9-NC_000913_3gb-stationary-37-Gluco...,13 25 0 1,3041632,SNP,T→A,V107E (GTG→GAG),ygfZ,private,True,{ygfZ},{b2898},{b2898}
468720,GLU_16321-M9-NC_000913_3gb-stationary-37-Gluco...,17 23 0 1,3041632,SNP,T→A,V107E (GTG→GAG),ygfZ,private,True,{ygfZ},{b2898},{b2898}
468758,GLU_16321-M9-NC_000913_3gb-stationary-37-Gluco...,19 21 0 1,3041632,SNP,T→A,V107E (GTG→GAG),ygfZ,private,True,{ygfZ},{b2898},{b2898}
158708,GLU_18361-M9-NC_000913_3gb-stationary-37-Gluco...,21 21 0 1,3041634,SNP,A→C,T108P (ACC→CCC),ygfZ,private,True,{ygfZ},{b2898},{b2898}


In [19]:
# !!! Temporary; need to reprocess all ALE experiment data for this project with the now removed ERS project experiments which were being duplicated by the newer ROS_eAMP ALE project/experiment.

meta_muts = meta_muts[meta_muts.exp != 'GLU_16321-M9-NC_000913_3gb-stationary-37-Glucose2-NH4Cl1-KH2PO43-MgSO4024-O2']
meta_muts = meta_muts[meta_muts.exp != 'GLU_18361-M9-NC_000913_3gb-stationary-37-Glucose2-NH4Cl1-KH2PO43-MgSO4024-O2']
meta_muts = meta_muts[meta_muts.exp != 'GLU_4721-M9-NC_000913_3gb-stationary-37-Glucose2-NH4Cl1-KH2PO43-MgSO4024-O2']
meta_muts = meta_muts[meta_muts.exp != 'OxidizeME_PQ3']

In [20]:
import os, sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
from aledbmutil.mut import get_mut_size, get_mob_size, is_genetic_mut, is_coding_mut, predict_mutation_effect_on_feature, get_SNP_aa_pos, get_DEL_INS_MOB_aa_start_pos, get_DEL_AA_range


meta_muts["genetic"] = meta_muts["Details"].apply(is_genetic_mut)
meta_muts["coding"] = meta_muts["Details"].apply(is_coding_mut)
meta_muts["mutation size"] = meta_muts.apply(get_mut_size, axis=1)  # get_mut_size currently doesn't return MOB mut sizes. Isn't something currently necessary.

In [21]:
from aledbmutil.mut import get_SNP_aa_pos, get_DEL_AA_range

def _get_SNP_AA_range(mut_details_str):
    SNP_AA_range = ()
    AA_pos = get_SNP_aa_pos(mut_details_str)
    SNP_AA_range = (AA_pos, AA_pos)
    return SNP_AA_range

meta_muts["AA range"] = None
# meta_muts["AA range"] = meta_muts.apply(
#     lambda r: _get_SNP_AA_range(r["Details"]) if ((r["Mutation Type"] == "SNP") & (r["coding"]==1)) else r["AA range"],
#     axis=1)
meta_muts["AA range"] = meta_muts.apply(lambda r:
                                        _get_SNP_AA_range(r["Details"]) if ((r["Mutation Type"] == "SNP") & (r["coding"]==1)) \
                                        else (get_DEL_AA_range(r["Details"]) if ((r["Mutation Type"] == "DEL") & (r["coding"]==1)) \
                                        else r["AA range"]),
                                        axis=1)
meta_muts.head()

,exp,sample,Position,Mutation Type,Sequence Change,Details,Gene,ALEdb source,genetic,clean genes,bnums,genes of interest,coding,mutation size,AA range
159621,GLU_ALE4-M9-NC_000913_3gb-stationary-37-Glucos...,14 21 0 1,3041398,SNP,T→G,L29R (CTT→CGT),ygfZ,private,True,{ygfZ},{b2898},{b2898},True,1,"(29, 29)"
159668,GLU_ALE4-M9-NC_000913_3gb-stationary-37-Glucos...,16 28 1 1,3041632,SNP,T→A,V107E (GTG→GAG),ygfZ,private,True,{ygfZ},{b2898},{b2898},True,1,"(107, 107)"
159706,GLU_ALE4-M9-NC_000913_3gb-stationary-37-Glucos...,18 24 1 1,3041393,SNP,G→T,W27C (TGG→TGT),ygfZ,private,True,{ygfZ},{b2898},{b2898},True,1,"(27, 27)"
159725,GLU_ALE4-M9-NC_000913_3gb-stationary-37-Glucos...,18 36 1 1,3041634,SNP,A→C,T108P (ACC→CCC),ygfZ,private,True,{ygfZ},{b2898},{b2898},True,1,"(108, 108)"
159693,GLU_ALE4-M9-NC_000913_3gb-stationary-37-Glucos...,18 11 1 1,3041973,SNP,G→C,G221R (GGT→CGT),ygfZ,private,True,{ygfZ},{b2898},{b2898},True,1,"(221, 221)"


In [22]:
# currently only tracking seq changes with SNPs for ALEdb mutations for later processing with AA property change trends
meta_muts['AA ref seq'] = meta_muts.apply(lambda m: m['Details'][0] if m["Mutation Type"]=='SNP' else '', axis=1)
meta_muts['AA seq change'] = meta_muts.apply(lambda m: m['Details'].split(' (')[0][-1] if m["Mutation Type"]=='SNP' else '', axis=1)
meta_muts.head()

,exp,sample,Position,Mutation Type,Sequence Change,Details,Gene,ALEdb source,genetic,clean genes,bnums,genes of interest,coding,mutation size,AA range,AA ref seq,AA seq change
159621,GLU_ALE4-M9-NC_000913_3gb-stationary-37-Glucos...,14 21 0 1,3041398,SNP,T→G,L29R (CTT→CGT),ygfZ,private,True,{ygfZ},{b2898},{b2898},True,1,"(29, 29)",L,R
159668,GLU_ALE4-M9-NC_000913_3gb-stationary-37-Glucos...,16 28 1 1,3041632,SNP,T→A,V107E (GTG→GAG),ygfZ,private,True,{ygfZ},{b2898},{b2898},True,1,"(107, 107)",V,E
159706,GLU_ALE4-M9-NC_000913_3gb-stationary-37-Glucos...,18 24 1 1,3041393,SNP,G→T,W27C (TGG→TGT),ygfZ,private,True,{ygfZ},{b2898},{b2898},True,1,"(27, 27)",W,C
159725,GLU_ALE4-M9-NC_000913_3gb-stationary-37-Glucos...,18 36 1 1,3041634,SNP,A→C,T108P (ACC→CCC),ygfZ,private,True,{ygfZ},{b2898},{b2898},True,1,"(108, 108)",T,P
159693,GLU_ALE4-M9-NC_000913_3gb-stationary-37-Glucos...,18 11 1 1,3041973,SNP,G→C,G221R (GGT→CGT),ygfZ,private,True,{ygfZ},{b2898},{b2898},True,1,"(221, 221)",G,R


In [23]:
exp_sel_pres_d = {
    'ROS_eAMP': "paraquat",  # starting strains were also pre-evolved on glucose
 }

meta_muts['selection pressure'] = meta_muts.apply(lambda r: 'unpublished' if r['ALEdb source'] == 'private' else '', axis=1)
meta_muts['selection pressure'] = meta_muts.exp.apply(lambda e: exp_sel_pres_d[e] if e in exp_sel_pres_d.keys() else 'unpublished')
assert('' not in set(meta_muts['selection pressure']))

In [24]:
# TODO: fix warnings with append
import warnings
warnings.filterwarnings('ignore')


aa_var_df = pd.DataFrame()

from aledbmutil.mut import get_mut_size  #, get_DEL_INS_MOB_aa_start_pos


for mt, mut_type_mut_df in meta_muts.groupby("Mutation Type"):
    assert mt != "MOB", 'Currently not handilng MOB. The below INS logic might be able to handle this.'
    if mt == "SNP":
        for _, m in mut_type_mut_df.iterrows():
            mut_eff = ""
            details_str = m["Details"]
            details_str = details_str[0:details_str.find(" (")]
            
            if details_str[-1] == '*':  # Could have them show the effected downstream positions like with ACS SynBio paper. 
                mut_eff = "truncation"
            elif details_str[0] == details_str[-1]:
                mut_eff = "synonymous substitution"
            elif details_str[0] != details_str[-1]:
                mut_eff = "nonsynonymous substitution"
            else:
                mut_eff = "unknown"
            
            aa_pos = int(details_str[1:-1])
            aa_var_df= aa_var_df.append({
                "AA pos": aa_pos,
                "mutation type": "SUB",
                "mutation effect": mut_eff,
                "mutation source": m['selection pressure'],
                'AA ref seq': m['AA ref seq'],
                'AA seq change': m['AA seq change'],
            }, ignore_index=True)
    if (mt == 'DEL'):  # Unravel since downstream analysis just looks at individual mutated AAs no matter if they come from the same allele.
        for _, m in mut_type_mut_df.iterrows():
            mut_eff = "deletion"  # currently assuming only insertions remain
            mut_size = m['mutation size']
            if mut_size % 3 != 0:
                assert mut_size % 3 != 0, 'not dealing with frameshifts yet' # Could have them show the effected downstream positions like with ACS SynBio paper.
            else:
                for aa_pos in range(m['AA range'][0], m['AA range'][1]+1):
                    aa_var_df= aa_var_df.append({
                        "AA pos": aa_pos,
                        "mutation type": mt,
                        "mutation effect": mut_eff,
                        "mutation source": m['selection pressure'],
                        'AA ref seq': '',
                        'AA seq change': '',
                    },ignore_index=True)
    # if (mt == 'INS'):
    #     for _, m in mut_type_mut_df.iterrows():
    #         mut_eff = "insertion"  # currently assuming only insertions remain
    #         mut_size = get_mut_size(m)
    #         if mut_size % 3 != 0:
    #             mut_eff = "frameshift"
    #         aa_var_df= aa_var_df.append({
    #             "AA pos": get_DEL_INS_MOB_aa_start_pos(m["Details"]),
    #             "mutation type": mt,
    #             "mutation effect": mut_eff,
    #             "mutation source": m['selection pressure'],
    #             'AA ref seq': '',
    #             'AA seq change': '',
    #         },ignore_index=True)
    if (mt == 'SUB'):  # Unravel since downstream analysis just looks at individual mutated AAs no matter if they come from the same allele.
        for _, m in mut_type_mut_df.iterrows():
            for aa_pos in range(m['AA range'][0], m['AA range'][1]+1):
                aa_var_df= aa_var_df.append({
                    "AA pos": aa_pos,
                    "mutation type": mt,
                    "mutation effect": "nonsynonymous substitution",
                    "mutation source": m['selection pressure'],
                    'AA ref seq': '',
                    'AA seq change': '',
                },ignore_index=True)


display(len(aa_var_df), aa_var_df.head())

31

,AA pos,mutation type,mutation effect,mutation source,AA ref seq,AA seq change
0,29,SUB,nonsynonymous substitution,unpublished,L,R
1,107,SUB,nonsynonymous substitution,unpublished,V,E
2,27,SUB,nonsynonymous substitution,unpublished,W,C
3,108,SUB,nonsynonymous substitution,unpublished,T,P
4,221,SUB,nonsynonymous substitution,unpublished,G,R


In [25]:
aa_var_cnt_df = pd.DataFrame()
gcols = ['AA pos', 'mutation type', 'mutation effect', 'mutation source', 'AA ref seq', 'AA seq change']
for g, gdf in aa_var_df.groupby(gcols):  # Essentially just finding duplicates
    d = dict(zip(gcols, g))
    d['mutation count'] = len(gdf)
    aa_var_cnt_df = aa_var_cnt_df.append(d, ignore_index=True)
aa_var_cnt_df.head()

,AA pos,mutation type,mutation effect,mutation source,AA ref seq,AA seq change,mutation count
0,27,SUB,nonsynonymous substitution,paraquat,W,C,1
1,27,SUB,nonsynonymous substitution,unpublished,W,C,1
2,29,SUB,nonsynonymous substitution,paraquat,L,R,6
3,29,SUB,nonsynonymous substitution,unpublished,L,R,1
4,46,SUB,nonsynonymous substitution,unpublished,T,I,1


In [26]:
import os, sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
import const


aa_var_cnt_df['mutation effect color'] = aa_var_cnt_df['mutation effect'].apply(lambda e: const.MUTATION_TYPE_COLOR_PALETTE[e])
aa_var_cnt_df.head()

,AA pos,mutation type,mutation effect,mutation source,AA ref seq,AA seq change,mutation count,mutation effect color
0,27,SUB,nonsynonymous substitution,paraquat,W,C,1,#DD8452
1,27,SUB,nonsynonymous substitution,unpublished,W,C,1,#DD8452
2,29,SUB,nonsynonymous substitution,paraquat,L,R,6,#DD8452
3,29,SUB,nonsynonymous substitution,unpublished,L,R,1,#DD8452
4,46,SUB,nonsynonymous substitution,unpublished,T,I,1,#DD8452


In [27]:
set(aa_var_cnt_df['mutation source'])

{'paraquat', 'unpublished'}

In [28]:
sel_pres_clr_d = {
    # 'FeSO4': '#729ECE',
    # 'Homo sapiens tpi': '#FF9E4A',
    # 'Pyrobaculum aerophilum pgi': '#67BF5C',
    # 'Pyrobaculum aerophilum tpi': '#ED665D',
    # 'Vibrio cholerae tpi': '#AD8BC9',
    # 'isobutanol': '#ED97CA',
    'paraquat': '#6DCCDA',
    'unpublished': '#AD8BC9'
 }

aa_var_cnt_df['selection pressure color'] = aa_var_cnt_df['mutation source'].apply(lambda e: sel_pres_clr_d[e])
aa_var_cnt_df

,AA pos,mutation type,mutation effect,mutation source,AA ref seq,AA seq change,mutation count,mutation effect color,selection pressure color
0,27,SUB,nonsynonymous substitution,paraquat,W,C,1,#DD8452,#6DCCDA
1,27,SUB,nonsynonymous substitution,unpublished,W,C,1,#DD8452,#AD8BC9
2,29,SUB,nonsynonymous substitution,paraquat,L,R,6,#DD8452,#6DCCDA
3,29,SUB,nonsynonymous substitution,unpublished,L,R,1,#DD8452,#AD8BC9
4,46,SUB,nonsynonymous substitution,unpublished,T,I,1,#DD8452,#AD8BC9
5,61,SUB,synonymous substitution,unpublished,A,A,1,#55A868,#AD8BC9
6,107,SUB,nonsynonymous substitution,paraquat,V,E,5,#DD8452,#6DCCDA
7,107,SUB,nonsynonymous substitution,unpublished,V,E,1,#DD8452,#AD8BC9
8,108,SUB,nonsynonymous substitution,paraquat,T,P,6,#DD8452,#6DCCDA
9,108,SUB,nonsynonymous substitution,unpublished,T,P,1,#DD8452,#AD8BC9


In [29]:
def _get_variant_name(variant):
    var_name = ''
    if variant['mutation type'] == 'SUB':
        var_name = variant['mutation type'] + " " + variant['AA ref seq'] + str(variant['AA pos']) + variant['AA seq change']
    return var_name


aa_var_cnt_df['name'] = aa_var_cnt_df.apply(lambda r: _get_variant_name(r), axis=1)
aa_var_cnt_df

,AA pos,mutation type,mutation effect,mutation source,AA ref seq,AA seq change,mutation count,mutation effect color,selection pressure color,name
0,27,SUB,nonsynonymous substitution,paraquat,W,C,1,#DD8452,#6DCCDA,SUB W27C
1,27,SUB,nonsynonymous substitution,unpublished,W,C,1,#DD8452,#AD8BC9,SUB W27C
2,29,SUB,nonsynonymous substitution,paraquat,L,R,6,#DD8452,#6DCCDA,SUB L29R
3,29,SUB,nonsynonymous substitution,unpublished,L,R,1,#DD8452,#AD8BC9,SUB L29R
4,46,SUB,nonsynonymous substitution,unpublished,T,I,1,#DD8452,#AD8BC9,SUB T46I
5,61,SUB,synonymous substitution,unpublished,A,A,1,#55A868,#AD8BC9,SUB A61A
6,107,SUB,nonsynonymous substitution,paraquat,V,E,5,#DD8452,#6DCCDA,SUB V107E
7,107,SUB,nonsynonymous substitution,unpublished,V,E,1,#DD8452,#AD8BC9,SUB V107E
8,108,SUB,nonsynonymous substitution,paraquat,T,P,6,#DD8452,#6DCCDA,SUB T108P
9,108,SUB,nonsynonymous substitution,unpublished,T,P,1,#DD8452,#AD8BC9,SUB T108P


In [30]:
aa_var_cnt_df.to_csv('ygfZ_ale_muts.csv', index=False)

In [31]:
mut_needle_plot_legend_df = pd.DataFrame()
mut_needle_plot_legend_df = aa_var_cnt_df.copy()
mut_needle_plot_legend_df = mut_needle_plot_legend_df[['mutation source', 'selection pressure color', ]]
mut_needle_plot_legend_df.rename(columns={'mutation source': 'name', 'selection pressure color': 'fill'}, inplace=True)
mut_needle_plot_legend_df['edge'] = 'white'
mut_needle_plot_legend_df.drop_duplicates(inplace=True)

df = aa_var_cnt_df.copy()
df = df[['mutation effect', 'mutation effect color']]
df.rename(columns={'mutation effect': 'name', 'mutation effect color': 'edge'}, inplace=True)
df.drop_duplicates(inplace=True)
df['fill'] = 'white'
mut_needle_plot_legend_df = mut_needle_plot_legend_df.append(df, ignore_index=True)
display(mut_needle_plot_legend_df)

mut_needle_plot_legend_df.to_csv('ygfZ_ale_muts_legend.csv', index=False)

,name,fill,edge
0,paraquat,#6DCCDA,white
1,unpublished,#AD8BC9,white
2,nonsynonymous substitution,white,#DD8452
3,synonymous substitution,white,#55A868


In [32]:
feats_df = pd.read_pickle('./annot_df.pkl')

# filtering out features that aren't useful
feats_to_remove = [
    # 'Chain',
    'Initiator methionine',
    'tRNA-modifying protein YgfZ [ygfZ]',
    'tRNA-modifying protein YgfZ',
    'folate-binding protein YgfZ',
    'UNCHARACTERIZED',
    'Aminomethyltransferase, folate-binding domain',  # redundant
    'YgfZ/GcvT conserved site',
    'YgfZ/GcvT',
    'Aminomethyltransferase beta-barrel domains',
    'Aminomethyltransferase beta-barrel domain',
]
feats_df = feats_df[~(feats_df['feature'].isin(feats_to_remove))]
feats_df.reset_index(inplace=True, drop=True)
feats_df

,feature,start,end,range
0,Chain,2,326,"(2, 325)"
1,"Glycine cleavage T-protein/YgfZ, C-terminal",244,325,"(244, 82)"
2,Folate-binding domain,6,242,"(6, 237)"
3,Folate binding site,189,189,"(189, 1)"
4,Aminomethyltransferase folate-binding domain,31,135,"(31, 105)"
5,Folate binding site,27,27,"(27, 1)"
6,GcvT family signature motif,187,251,"(187, 65)"
7,Acetate binding site,27,27,"(27, 1)"
8,Acetate binding site,74,74,"(74, 1)"
9,Acetate binding site,162,162,"(162, 1)"


In [33]:
layer = 0
layer_d = dict()
for f in feats_df.feature.unique():
    layer_d[f] = layer
    layer += 1
feats_df['layer'] = feats_df['feature'].apply(lambda f: layer_d[f])
feats_df.head()

,feature,start,end,range,layer
0,Chain,2,326,"(2, 325)",0
1,"Glycine cleavage T-protein/YgfZ, C-terminal",244,325,"(244, 82)",1
2,Folate-binding domain,6,242,"(6, 237)",2
3,Folate binding site,189,189,"(189, 1)",3
4,Aminomethyltransferase folate-binding domain,31,135,"(31, 105)",4


In [34]:
len(feats_df.feature.unique())

11

In [35]:
feat_color_d = dict()
# muted=["#4878D0", "#EE854A", "#6ACC64", "#D65F5F", "#956CB4", "#8C613C", "#DC7EC0", "#797979", "#D5BB67", "#82C6E2"]  # from seaborn
clrs = ['#7F3C8D',
'#11A579',
'#3969AC',
'#F2B701',
'#E73F74',
'#80BA5A',
'#E68310',
'#008695',
'#CF1C90',
'#F97B72',
'#A5AA99']
feats = list(feats_df.feature.unique())
for i in range(0, len(feats_df.feature.unique())):
    feat_color_d[feats[i]] = clrs[i]
feat_color_d

feats_df['color'] = feats_df['feature'].apply(lambda f: feat_color_d[f])

feats_df.drop(columns=["range"], inplace=True)
feats_df.drop_duplicates(inplace=True)

display(feats_df, feat_color_d)
feats_df.to_csv('./ygfZ_feats.csv')

,feature,start,end,layer,color
0,Chain,2,326,0,#7F3C8D
1,"Glycine cleavage T-protein/YgfZ, C-terminal",244,325,1,#11A579
2,Folate-binding domain,6,242,2,#3969AC
3,Folate binding site,189,189,3,#F2B701
4,Aminomethyltransferase folate-binding domain,31,135,4,#E73F74
5,Folate binding site,27,27,3,#F2B701
6,GcvT family signature motif,187,251,5,#80BA5A
7,Acetate binding site,27,27,6,#E68310
8,Acetate binding site,74,74,6,#E68310
9,Acetate binding site,162,162,6,#E68310


{'Chain': '#7F3C8D',
 'Glycine cleavage T-protein/YgfZ, C-terminal': '#11A579',
 'Folate-binding domain': '#3969AC',
 'Folate binding site': '#F2B701',
 'Aminomethyltransferase folate-binding domain': '#E73F74',
 'GcvT family signature motif': '#80BA5A',
 'Acetate binding site': '#E68310',
 'Ca binding site': '#008695',
 'Cl binding site': '#CF1C90',
 'Ethandiol binding site': '#F97B72',
 'SO4 binding site': '#A5AA99'}

In [36]:
# Filtering out some features that are not useful for the NGL visual 3D analysis
ngl_feat_df = feats_df.copy()
ngl_feat_df = ngl_feat_df[ngl_feat_df['feature'].isin(['Ethandiol binding site','GcvT family signature motif','Aminomethyltransferase folate-binding domain'])]

In [37]:
# for NGL
ngl_feat_df["size"] = ngl_feat_df.apply(lambda r: r["end"] - r["start"] + 1, axis=1)
ngl_feat_df = ngl_feat_df[["feature", "start", "end", "color", "size"]]
ngl_feat_df = ngl_feat_df.sort_values("size", ascending=False)
ngl_feat_df = ngl_feat_df.reset_index(drop=True)  # json explicitly uses the indexes

with open('feats.json', 'w') as outfile:   # for NGL
    outfile.write(ngl_feat_df.to_json())
ngl_feat_df

,feature,start,end,color,size
0,Aminomethyltransferase folate-binding domain,31,135,#E73F74,105
1,GcvT family signature motif,187,251,#80BA5A,65
2,Ethandiol binding site,23,25,#F97B72,3
3,Ethandiol binding site,107,109,#F97B72,3
4,Ethandiol binding site,116,117,#F97B72,2
5,Ethandiol binding site,16,16,#F97B72,1
6,Ethandiol binding site,38,38,#F97B72,1
7,Ethandiol binding site,50,50,#F97B72,1
8,Ethandiol binding site,99,99,#F97B72,1
9,Ethandiol binding site,260,260,#F97B72,1
